## Загрузка данных

In [1]:
import re
import spacy
import numpy as np
import pandas as pd
from datasets import load_dataset

from dataclasses import dataclass
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import chain
from typing import List, Dict, Tuple
from collections import Counter, defaultdict

In [2]:
def load_translation_dataset():
    print("Loading Tatoeba en-ru...")
    try:
        dataset = load_dataset("Helsinki-NLP/tatoeba", lang1="en", lang2="ru", trust_remote_code=True)
        
    except Exception as e:
        print(f"Error while loading dataset: {e}")
        raise
    
    print("\nDataset structure:")
    print(dataset)
    
    print("\nData sample:")
    for i in range(2):
        print(f"EN: {dataset['train'][i]['translation']['en']}")
        print(f"RU: {dataset['train'][i]['translation']['ru']}\n")

    return dataset

In [3]:
dataset = load_translation_dataset()

Loading Tatoeba en-ru...


README.md:   0%|          | 0.00/8.93k [00:00<?, ?B/s]

tatoeba.py:   0%|          | 0.00/4.41k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]


Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

Data sample:
EN: For once in my life I'm doing a good deed... And it is useless.
RU: Один раз в жизни я делаю хорошее дело... И оно бесполезно.

EN: Let's try something.
RU: Давайте что-нибудь попробуем!



In [4]:
from transformers import AutoTokenizer
import numpy as np

def prepare_data_with_hf(
    dataset,
    model_name: str = "Helsinki-NLP/opus-mt-en-ru",
    max_length: int = 128,
    batch_size: int = 32
):
    # Инициализация и настройка токенизатора
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Принудительное добавление специальных токенов
    special_tokens = {
        'bos_token': '<s>',
        'eos_token': '</s>',
        'pad_token': '<pad>',
        'unk_token': '<unk>'
    }
    tokenizer.add_special_tokens(special_tokens)
    
    # Принудительная установка атрибутов
    tokenizer.bos_token = '<s>'
    tokenizer.eos_token = '</s>'
    tokenizer.pad_token = '<pad>'
    tokenizer.unk_token = '<unk>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<s>')
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('</s>')
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<pad>')

    def preprocess_function(examples):
        # Обработка исходных текстов с BOS
        source_texts = [
            f"{tokenizer.bos_token} {item['en']}"  # Явное добавление BOS
            for item in examples['translation']
        ]
        
        # Обработка целевых текстов с BOS/EOS
        target_texts = [
            f"{tokenizer.bos_token} {item['ru']} {tokenizer.eos_token}"
            for item in examples['translation']
        ]

        # Токенизация исходных текстов
        source_encoding = tokenizer(
            source_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np',
            add_special_tokens=False  # Токены уже добавлены вручную
        )
        
        # Токенизация целевых текстов
        target_encoding = tokenizer(
            target_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors='np',
            add_special_tokens=False
        )

        # Принудительная проверка и коррекция BOS/EOS
        # Для исходных последовательностей
        source_ids = source_encoding['input_ids']
        source_ids[:, 0] = tokenizer.bos_token_id  # Гарантируем BOS в начале
        
        # Для целевых последовательностей
        target_ids = target_encoding['input_ids']
        target_ids[:, 0] = tokenizer.bos_token_id  # BOS в начале
        
        # Находим позиции для EOS
        target_attention_mask = target_encoding['attention_mask']
        seq_lens = np.argmin(target_attention_mask, axis=1)
        seq_lens[seq_lens == 0] = target_attention_mask.shape[1]
        
        # Устанавливаем EOS
        for i, pos in enumerate(seq_lens):
            if pos < max_length:
                target_ids[i, pos-1] = tokenizer.eos_token_id
            else:
                target_ids[i, -1] = tokenizer.eos_token_id

        return {
            'input_ids': source_ids,
            'attention_mask': source_encoding['attention_mask'],
            'labels': target_ids,
            'decoder_attention_mask': target_encoding['attention_mask']
        }
    
    processed_dataset = dataset['train'].map(
        preprocess_function,
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset['train'].column_names
    )
    
    return processed_dataset, tokenizer

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [5]:
processed_data, hf_tokenizer = prepare_data_with_hf(dataset)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/523656 [00:00<?, ? examples/s]

In [6]:
sample = processed_data[0]
print("Source:", hf_tokenizer.decode(sample['input_ids']))
print("Target:", hf_tokenizer.decode(sample['labels']))

Source: <s>  For once in my life I'm doing a good deed... And it is useless.<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
Target: <s> Один раз в жизни я делаю хорошее дело... И оно бесполезно.</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

## Архитектура модели

In [7]:
%%capture
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

In [8]:
import os
import time
import math
import copy
import spacy
import GPUtil
import pandas as pd
from typing import *
from itertools import chain

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.optim.lr_scheduler import LambdaLR
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, Dataset

import altair as alt
from altair import Chart

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach()
        return self.dropout(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Q: [batch_size, num_heads, seq_len, head_dim]
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        
        attn_probs = F.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)
        
        # Линейные преобразования
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Вычисление внимания
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Объединение голов
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # Финальное линейное преобразование
        output = self.W_o(attn_output)
        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, mask=None):
        # Self attention
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForward(d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        # Self attention (маскированное)
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        # Cross attention (с выходом энкодера)
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        
        # Feed forward
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        return x

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, num_layers=6):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout=0.1)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads) for _ in range(num_layers)])
        
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
            
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Проверка размерностей входных данных
        batch_size = src.size(0)
        src_seq_len = src.size(1)
        tgt_seq_len = tgt.size(1)
        
        # Энкодинг
        src_emb = self.positional_encoding(self.encoder_embedding(src))
        enc_output = src_emb
        
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)
        
        # Декодинг
        tgt_emb = self.positional_encoding(self.decoder_embedding(tgt))
        dec_output = tgt_emb
        
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, src_mask, tgt_mask)
        
        # Финальный слой
        output = self.fc_out(dec_output)
        return output

In [10]:
def test_transformer():
    # Проверяем доступность CUDA и инициализируем устройство
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Установка seed для воспроизводимости
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    torch.manual_seed(42)
    
    # Конфигурация
    batch_size = 2
    seq_len = 10
    d_model = 512
    num_heads = 8
    src_vocab_size = 100
    tgt_vocab_size = 100
    num_layers = 2

    # Генерация синтетических данных
    src = torch.randint(0, src_vocab_size, (batch_size, seq_len)).to(device)
    tgt = torch.randint(0, tgt_vocab_size, (batch_size, seq_len)).to(device)
    
    # Генерация масок
    src_mask = torch.ones(batch_size, 1, 1, seq_len).to(device)
    tgt_mask = torch.tril(torch.ones(seq_len, seq_len)).expand(batch_size, 1, seq_len, seq_len).to(device)

    # Инициализация модели
    transformer = Transformer(
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    ).to(device)

    print("="*50)
    print("1. Тест Positional Encoding")
    pe = PositionalEncoding(d_model, dropout=0.1).to(device)
    x = torch.randn(1, seq_len, d_model).to(device)
    print(f"До PE: mean={x.mean().item():.4f}, std={x.std().item():.4f}")
    x_pe = pe(x)
    print(f"После PE: mean={x_pe.mean().item():.4f}, std={x_pe.std().item():.4f}")
    print(f"Форма PE: {x_pe.shape} (должна быть [1, {seq_len}, {d_model}])")
    
    print("\n2. Тест Multi-Head Attention")
    mha = MultiHeadAttention(d_model, num_heads).to(device)
    q = k = v = torch.randn(batch_size, seq_len, d_model).to(device)
    attn_output = mha(q, k, v)
    print(f"Форма выхода внимания: {attn_output.shape} (должна быть {q.shape})")
    print(f"Максимальное значение: {attn_output.max().item():.4f}")
    print(f"Минимальное значение: {attn_output.min().item():.4f}")

    print("\n3. Тест Encoder Layer")
    encoder_layer = EncoderLayer(d_model, num_heads).to(device)
    enc_input = torch.randn(batch_size, seq_len, d_model).to(device)
    enc_output = encoder_layer(enc_input)
    print(f"Форма выхода энкодера: {enc_output.shape} (должна быть {enc_input.shape})")
    print(f"Изменение данных: {torch.allclose(enc_input, enc_output, atol=1e-4)} (должно быть False)")

    print("\n4. Тест Decoder Layer")
    decoder_layer = DecoderLayer(d_model, num_heads).to(device)
    dec_input = torch.randn(batch_size, seq_len, d_model).to(device)
    dec_output = decoder_layer(dec_input, enc_output, src_mask, tgt_mask)
    print(f"Форма выхода декодера: {dec_output.shape} (должна быть {dec_input.shape})")
    print(f"Норма выходных данных: {dec_output.norm().item():.4f}")

    print("\n5. Полный тест Transformer")
    print("Входные данные:")
    print(f"src: {src.shape} (max={src.max().item()}, min={src.min().item()})")
    print(f"tgt: {tgt.shape} (max={tgt.max().item()}, min={tgt.min().item()})")
    
    output = transformer(src, tgt, src_mask, tgt_mask)
    print("\nПроверка формы выхода:")
    print(f"Ожидаемая форма: ({batch_size}, {seq_len}, {tgt_vocab_size})")
    print(f"Реальная форма:   {output.shape}")
    
    print("\nПроверка градиентов:")
    dummy_loss = output.sum()
    dummy_loss.backward()
    has_gradients = any(p.grad is not None for p in transformer.parameters())
    print(f"Градиенты вычислены: {has_gradients} (должно быть True)")

    print("\n6. Проверка параметров модели:")
    total_params = sum(p.numel() for p in transformer.parameters())
    print(f"Всего параметров: {total_params}")
    print(f"Параметры энкодера: {sum(p.numel() for p in transformer.encoder_embedding.parameters())}")
    print(f"Параметры декодера: {sum(p.numel() for p in transformer.decoder_embedding.parameters())}")

    print("\nТест завершен!")

In [11]:
test_transformer()

Using device: cuda
1. Тест Positional Encoding
До PE: mean=-0.0316, std=1.0065
После PE: mean=0.4401, std=1.2013
Форма PE: torch.Size([1, 10, 512]) (должна быть [1, 10, 512])

2. Тест Multi-Head Attention
Форма выхода внимания: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Максимальное значение: 0.3949
Минимальное значение: -0.4440

3. Тест Encoder Layer
Форма выхода энкодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Изменение данных: False (должно быть False)

4. Тест Decoder Layer
Форма выхода декодера: torch.Size([2, 10, 512]) (должна быть torch.Size([2, 10, 512]))
Норма выходных данных: 101.1924

5. Полный тест Transformer
Входные данные:
src: torch.Size([2, 10]) (max=95, min=6)
tgt: torch.Size([2, 10]) (max=99, min=10)

Проверка формы выхода:
Ожидаемая форма: (2, 10, 100)
Реальная форма:   torch.Size([2, 10, 100])

Проверка градиентов:
Градиенты вычислены: True (должно быть True)

6. Проверка параметров модели:
Всего параметров: 14866532
Па

In [12]:
def translate_sentence(model, tokenizer, sentence, device, max_length=128):
    model.eval()
    
    # Добавляем BOS токен вручную
    inputs = tokenizer(
        f"{tokenizer.bos_token} {sentence}",
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)
    
    # Инициализируем декодер с BOS токена
    decoder_input = torch.tensor([[tokenizer.bos_token_id]], device=device)
    
    with torch.no_grad():
        for _ in range(max_length):
            # Создаем маски
            src_mask = (inputs['input_ids'] != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(decoder_input.size(1), decoder_input.size(1), device=device)).bool()
            
            # Forward pass
            output = model(
                inputs['input_ids'],
                decoder_input,
                src_mask,
                tgt_mask.unsqueeze(0).unsqueeze(0)
            )
            
            # Берем последний токен
            next_token = output[:, -1].argmax(-1)
            
            # Добавляем к последовательности
            decoder_input = torch.cat([decoder_input, next_token.unsqueeze(1)], dim=1)
            
            # Останавливаемся при EOS
            if next_token.item() == tokenizer.eos_token_id:
                break
    
    # Декодируем результат, пропуская спецтокены
    return tokenizer.decode(decoder_input[0], skip_special_tokens=True)

## Тестирование

In [13]:
import torch
print(torch.__version__)  # Должно быть >= 1.11.0

2.5.1+cu121


In [14]:
def test_tokenization(tokenizer, test_sentence="Hello, how are you?"):
    print("="*50)
    print("1. Тестирование токенизации и специальных токенов")
    
    print(f"BOS: {tokenizer.bos_token} (id: {tokenizer.bos_token_id})")
    print(f"EOS: {tokenizer.eos_token} (id: {tokenizer.eos_token_id})")
    
    # Явное добавление BOS/EOS как в основном пайплайне
    processed_text = f"{tokenizer.bos_token} {test_sentence} {tokenizer.eos_token}"
    
    encoded = tokenizer(
        processed_text,
        padding='max_length',
        truncation=True,
        max_length=20,  # Уменьшаем для наглядности
        return_tensors='pt',
        add_special_tokens=False
    )
    
    token_ids = encoded['input_ids'][0].tolist()
    decoded = tokenizer.decode(token_ids)
    
    print("\nПример токенизации:")
    print(f"Токены: {token_ids}")
    print(f"Декодировано: {decoded}")
    
    # Находим позицию EOS
    try:
        eos_pos = token_ids.index(tokenizer.eos_token_id)
        pad_start = token_ids.index(tokenizer.pad_token_id)
        assert eos_pos < pad_start, "EOS должен быть перед паддингом"
    except ValueError:
        assert False, "EOS token not found"
    
    assert token_ids[0] == tokenizer.bos_token_id
    print("\n✅ Токенизация работает корректно")

def test_masking():
    print("\n" + "="*50)
    print("2. Тестирование создания масок")
    
    batch_size = 2
    seq_len = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Создаем тестовые данные
    src = torch.ones(batch_size, seq_len, device=device)
    tgt = torch.ones(batch_size, seq_len, device=device)
    
    # Маска энкодера (padding mask)
    src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
    
    # Маска декодера (padding + causal mask)
    causal_mask = torch.tril(torch.ones(seq_len, seq_len, device=device))
    tgt_mask = causal_mask.unsqueeze(0).repeat(batch_size, 1, 1, 1)
    
    print("\nSource mask (пример):")
    print(src_mask[0, 0, 0].cpu().numpy())  # Первый элемент батча
    
    print("\nTarget mask (пример):")
    print(tgt_mask[0, 0].cpu().numpy())     # Первый элемент батча
    
    # Проверка размерностей
    assert src_mask.shape == (batch_size, 1, 1, seq_len), f"Ожидалось (2,1,1,5), получено {src_mask.shape}"
    assert tgt_mask.shape == (batch_size, 1, seq_len, seq_len), f"Ожидалось (2,1,5,5), получено {tgt_mask.shape}"
    
    print("\n✅ Маски создаются корректно")

def test_model_forward_pass(model, tokenizer, device):
    print("\n" + "="*50)
    print("3. Тестирование forward pass модели")
    
    test_sentence = "Test input"
    inputs = tokenizer(
        test_sentence,
        return_tensors="pt",
        padding='max_length',
        max_length=128,
        truncation=True
    ).to(device)
    
    src = inputs['input_ids']
    tgt = torch.cat([
        torch.tensor([[tokenizer.bos_token_id]], device=device),
        src[:, :-1]
    ], dim=1)

    # Исправленное создание масок
    src_mask = inputs['attention_mask'].to(torch.bool).unsqueeze(1).unsqueeze(2)
    seq_len = tgt.size(1)
    tgt_mask = torch.tril(torch.ones(seq_len, seq_len, device=device)).to(torch.bool)
    tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        output = model(src, tgt, src_mask, tgt_mask)
    
    print("\nФорма выходов модели:", output.shape)
    print("Минимальное значение:", output.min().item())
    print("Максимальное значение:", output.max().item())
    
    assert not torch.isnan(output).any()
    expected_vocab_size = len(tokenizer)  # Используем реальный размер словаря
    assert output.shape == (1, 128, expected_vocab_size), (
        f"Ожидаемая форма: (1, 128, {expected_vocab_size}), "
        f"получено: {output.shape}"
    )
    print("\n✅ Forward pass работает корректно")

def test_dynamic_masking():
    print("Тестирование динамических масок для разных размеров")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_heads = 8
    d_model = 512
    
    # Тестовые параметры
    test_cases = [
        {'batch_size': 1, 'seq_len': 7},
        {'batch_size': 3, 'seq_len': 15},
        {'batch_size': 5, 'seq_len': 3}
    ]
    
    for case in test_cases:
        print(f"\nТест для batch={case['batch_size']} seq_len={case['seq_len']}")
        
        # Генерация масок
        src = torch.randint(0, 100, (case['batch_size'], case['seq_len'])).to(device)
        tgt = torch.randint(0, 100, (case['batch_size'], case['seq_len'])).to(device)
        
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = torch.tril(torch.ones(case['seq_len'], case['seq_len'])).to(device)
        tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
        
        # Проверка совместимости с Multi-Head Attention
        mha = MultiHeadAttention(d_model, num_heads).to(device)
        q = k = v = torch.randn(case['batch_size'], case['seq_len'], d_model).to(device)
        
        try:
            output = mha(q, k, v, tgt_mask)
            assert output.shape == (case['batch_size'], case['seq_len'], d_model)
            print("✅ Успех")
        except Exception as e:
            print(f"❌ Ошибка: {str(e)}")
            raise

def test_multi_head_compatibility():
    print("\nТестирование совместимости масок с Multi-Head Attention")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    d_model = 512
    num_heads_list = [4, 8, 16]
    seq_lens = [10, 15, 20]
    
    for num_heads in num_heads_list:
        for seq_len in seq_lens:
            print(f"\nHeads: {num_heads}, Seq_len: {seq_len}")
            
            # Создание масок
            mask = torch.tril(torch.ones(seq_len, seq_len)).to(device)
            
            # Проверка расширения масок
            mha = MultiHeadAttention(d_model, num_heads).to(device)
            q = torch.randn(2, seq_len, d_model).to(device)
            
            try:
                output = mha(q, q, q, mask)
                assert output.shape == q.shape
                print("✅ Совместимость подтверждена")
            except Exception as e:
                print(f"❌ Несовместимость: {str(e)}")
                raise

def test_translation_function(model, tokenizer, device):
    print("\n" + "="*50)
    print("4. Тестирование функции перевода")
    
    test_sentences = [
        "Hello world",
        "How are you?",
        "Test sentence",
        "Machine learning"
    ]
    
    for sentence in test_sentences:
        print("\n" + "-"*50)
        print(f"Перевод: '{sentence}'")
        
        try:
            translated = translate_sentence(
                model=model,
                tokenizer=tokenizer,
                sentence=sentence,
                device=device,
                max_length=64  # Уменьшенная длина для тестов
            )
            print(f"Результат: {translated}")
        except Exception as e:
            print(f"Ошибка: {str(e)}")
            raise
    
    print("\n✅ Функция перевода работает корректно")

def test_model_with_various_inputs():
    print("\nТестирование модели с различными входными размерами")
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    try:
        tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
        tokenizer.add_special_tokens({
            'bos_token': '<s>', 'eos_token': '</s>', 
            'pad_token': '<pad>', 'unk_token': '<unk>'
        })
    except Exception as e:
        print(f"Ошибка инициализации токенизатора: {e}")
        raise
    
    d_model = 512
    num_heads = 8
    num_layers = 6
    
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=d_model,
        num_heads=num_heads,
        num_layers=num_layers
    ).to(device)
    model.eval()
    
    test_cases = [
        {'batch_size': 1, 'src_len': 10, 'tgt_len': 12},
        {'batch_size': 3, 'src_len': 7, 'tgt_len': 9},
        {'batch_size': 5, 'src_len': 15, 'tgt_len': 15},
        {'batch_size': 2, 'src_len': 5, 'tgt_len': 5}
    ]
    
    for case in test_cases:
        print(f"\nТест: batch={case['batch_size']} src={case['src_len']} tgt={case['tgt_len']}")
        
        try:
            src = torch.randint(0, len(tokenizer), 
                             (case['batch_size'], case['src_len'])).to(device)
            tgt = torch.randint(0, len(tokenizer), 
                             (case['batch_size'], case['tgt_len'])).to(device)
            
            # Исправлено создание масок
            src_mask = (src != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = torch.tril(torch.ones(
                (case['batch_size'], case['tgt_len'], case['tgt_len']),
                device=device
            )).bool().unsqueeze(1)  # Добавлен unsqueeze для multi-head
            
            with torch.no_grad():
                output = model(src, tgt, src_mask, tgt_mask)
            
            expected_shape = (
                case['batch_size'], 
                case['tgt_len'], 
                len(tokenizer)
            )
            assert output.shape == expected_shape
            print("✅ Корректная работа")
            
        except Exception as e:
            print(f"❌ Ошибка: {str(e)}")
            raise

def run_comprehensive_tests():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Полная инициализация как в рабочем пайплайне
    tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")
    tokenizer.add_tokens(['<s>', '</s>', '<pad>', '<unk>'])
    tokenizer.bos_token = '<s>'
    tokenizer.eos_token = '</s>'
    tokenizer.pad_token = '<pad>'
    tokenizer.unk_token = '<unk>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<s>')
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('</s>')
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids('<pad>')
    
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=512,
        num_heads=8,
        num_layers=6
    ).to(device)
    
    # Запуск тестов
    test_tokenization(tokenizer)
    test_masking()
    test_model_forward_pass(model, tokenizer, device)
    test_dynamic_masking()
    test_multi_head_compatibility()
    test_model_with_various_inputs()
    test_translation_function(model, tokenizer, device)

# Запуск тестов
run_comprehensive_tests()

1. Тестирование токенизации и специальных токенов
BOS: <s> (id: 62518)
EOS: </s> (id: 0)

Пример токенизации:
Токены: [62518, 160, 5270, 2, 508, 55, 33, 19, 0, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517, 62517]
Декодировано: <s>  Hello, how are you?</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

✅ Токенизация работает корректно

2. Тестирование создания масок

Source mask (пример):
[ True  True  True  True  True]

Target mask (пример):
[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0.]
 [1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1.]]

✅ Маски создаются корректно

3. Тестирование forward pass модели

Форма выходов модели: torch.Size([1, 128, 62519])
Минимальное значение: -2.6944191455841064
Максимальное значение: 2.598710060119629

✅ Forward pass работает корректно
Тестирование динамических масок для разных размеров

Тест для batch=1 seq_len=7
✅ Успех

Тест для batch=3 seq_len=15
✅ Успех

Тест для batch=5 seq_len=3
✅ Успех

Тестирование совместим

## Обучение

In [15]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import Dataset

In [16]:
class TranslationDataset(Dataset):
    def __init__(self, processed_data):
        self.input_ids = processed_data['input_ids']
        self.attention_mask = processed_data['attention_mask']
        self.labels = processed_data['labels']
        self.decoder_attention_mask = processed_data['decoder_attention_mask']
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long),
            'decoder_attention_mask': torch.tensor(self.decoder_attention_mask[idx], dtype=torch.long)
        }

# Определение устройства
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
"""
# Полный код обучения
def train_model(subset_size=100, epochs=5, batch_size=32, lr=0.0001):
    # Загрузка и подготовка данных
    dataset = load_translation_dataset()
    processed_data, tokenizer = prepare_data_with_hf(dataset)
    
    # Создание подмножества
    train_subset = torch.utils.data.Subset(
        TranslationDataset(processed_data),
        indices=range(subset_size))
    
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    # Инициализация модели
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=512,
        num_heads=8,
        num_layers=6
    ).to(DEVICE)
    
    # Оптимизатор и функция потерь
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    
    # Цикл обучения
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            # Создание масок
            src_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            tgt_mask = (labels != tokenizer.pad_token_id).unsqueeze(1) & \
                      torch.tril(torch.ones(labels.size(1), labels.size(1))).bool().to(DEVICE)
            
            # Forward pass
            outputs = model(input_ids, labels[:, :-1], src_mask, tgt_mask[:, :-1, :-1])
            loss = criterion(outputs.view(-1, outputs.size(-1)), labels[:, 1:].contiguous().view(-1))
            
            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        # Вывод статистики и пример перевода
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")
        
        # Пример перевода
        test_sentence = "Hello, how are you?"
        translation = translate_sentence(model, tokenizer, test_sentence, DEVICE)
        print(f"\nExample translation: {translation}\n")
"""

'\n# Полный код обучения\ndef train_model(subset_size=100, epochs=5, batch_size=32, lr=0.0001):\n    # Загрузка и подготовка данных\n    dataset = load_translation_dataset()\n    processed_data, tokenizer = prepare_data_with_hf(dataset)\n    \n    # Создание подмножества\n    train_subset = torch.utils.data.Subset(\n        TranslationDataset(processed_data),\n        indices=range(subset_size))\n    \n    train_loader = DataLoader(\n        train_subset,\n        batch_size=batch_size,\n        shuffle=True,\n        num_workers=2,\n        pin_memory=True\n    )\n    \n    # Инициализация модели\n    model = Transformer(\n        src_vocab_size=len(tokenizer),\n        tgt_vocab_size=len(tokenizer),\n        d_model=512,\n        num_heads=8,\n        num_layers=6\n    ).to(DEVICE)\n    \n    # Оптимизатор и функция потерь\n    optimizer = Adam(model.parameters(), lr=lr)\n    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)\n    \n    # Цикл обучения\n    for epoc

In [17]:
def train_model(subset_size=100, epochs=5, batch_size=32, lr=0.0001):
    # Загрузка и подготовка данных
    dataset = load_translation_dataset()
    processed_data, tokenizer = prepare_data_with_hf(dataset)
    
    # Создание подмножества
    train_subset = torch.utils.data.Subset(
        TranslationDataset(processed_data),
        indices=range(subset_size))
    
    train_loader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    # Инициализация модели
    model = Transformer(
        src_vocab_size=len(tokenizer),
        tgt_vocab_size=len(tokenizer),
        d_model=512,
        num_heads=8,
        num_layers=6
    ).to(DEVICE)
    
    # Оптимизатор и функция потерь
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    
    # Цикл обучения
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            # Создание маски для энкодера
            src_mask = (input_ids != tokenizer.pad_token_id).unsqueeze(1).unsqueeze(2)
            
            # Создание маски для декодера с правильной размерностью
            seq_len = labels.size(1) - 1  # -1 потому что мы не используем последний токен в декодере
            tgt_mask = torch.tril(torch.ones((seq_len, seq_len), device=DEVICE)).bool()
            # Расширяем маску для batch и head dimensions
            tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
            
            # Forward pass с обрезанными последовательностями
            outputs = model(
                input_ids,
                labels[:, :-1],  # Убираем последний токен для входа декодера
                src_mask,
                tgt_mask
            )
            
            # Вычисление loss с правильным сдвигом
            loss = criterion(
                outputs.reshape(-1, outputs.size(-1)),
                labels[:, 1:].reshape(-1)  # Убираем первый токен для целевых значений
            )
            
            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        # Вывод статистики и пример перевода
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f}")
        
        # Пример перевода
        model.eval()
        with torch.no_grad():
            test_sentence = "Hello, how are you?"
            translation = translate_sentence(model, tokenizer, test_sentence, DEVICE)
            print(f"\nExample translation: {translation}\n")

In [18]:
# Запуск обучения на 100 примерах
train_model(subset_size=3000, epochs=30, batch_size=8, lr=0.0001)

Loading Tatoeba en-ru...

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

Data sample:
EN: For once in my life I'm doing a good deed... And it is useless.
RU: Один раз в жизни я делаю хорошее дело... И оно бесполезно.

EN: Let's try something.
RU: Давайте что-нибудь попробуем!



Epoch 1:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 1 | Loss: 3.1127

Example translation: Вы мол вы стелать вате ве вем ветемене ве сте ве вете вете ве верате вете веметете ве ве ве ве ве веме веле вемеме вемеме веме



Epoch 2:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 2 | Loss: 2.1144

Example translation: Вы должны вочитать восторите?



...

Epoch 28 | Loss: 0.1448

Example translation: У тебя несказать, как тебе свокой?



Epoch 29:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 29 | Loss: 0.1459

Example translation: Он спровёл, вы как себя верём?



Epoch 30:   0%|          | 0/375 [00:00<?, ?it/s]

Epoch 30 | Loss: 0.1367

Example translation: Он спросил, вы гороны?

